In [20]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import explained_variance_score,r2_score,mean_absolute_error
from sklearn import preprocessing

import numpy as np
import pandas as pd

from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, rand
import sys
import os
from utils import get_allstate_train_valid_test_testids
import h2o
import os
import time
h2o.init(max_mem_size = "40G")             #specify max number of bytes. uses all cores by default.
h2o.remove_all()  
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.glrm import H2OGeneralizedLowRankEstimator
from h2o import H2OFrame

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,3 days 3 hours 17 mins
H2O cluster version:,3.10.0.9
H2O cluster version age:,10 days
H2O cluster name:,H2O_from_python_arvc_lgnmyd
H2O cluster total nodes:,1
H2O cluster free memory:,35.31 Gb
H2O cluster total cores:,16
H2O cluster allowed cores:,16
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [21]:
#Read Input CSV file
shift = 203

train, valid, test, testids = get_allstate_train_valid_test_testids(0.15, shift, True)
test["loss"] = 0.0

trainframe = H2OFrame(train)
validframe = H2OFrame(valid)
testframe = H2OFrame(test)
del(train)
del(valid)
del(test)
y = "loss"
x = list(trainframe.columns)

Train shape is: (188318, 132)
Test shape is: (125546, 131)


/home/arvc/t81_558_deep_learning/utils.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  train.drop("type", axis=1, inplace=True)
/home/arvc/t81_558_deep_learning/utils.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test.drop("type", axis=1, inplace=True)


Final Train shape is: (160070, 1191)
Final Valid shape is: (28248, 1191)
Final Test shape is: (125546, 1191)
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [22]:
min_mae = 10000

def score(params):
    print("Training with params : ")
    print(params)
    
    model = H2OGradientBoostingEstimator(
                 ntrees=params["ntrees"],
                 learn_rate=params["learn_rate"],
                 max_depth=params["max_depth"],
                 sample_rate=params["sample_rate"],
                 col_sample_rate=0.7,
                 stopping_rounds=200,
                 stopping_tolerance=0.01, #10-fold increase in threshold as defined in rf_v1
                 score_each_iteration=True,
                 seed=200000,
                 nfolds=6,                 
                 keep_cross_validation_predictions=True
                )
    
   
    predictions = model.predict(validframe)
    score =  mean_absolute_error(np.exp(validframe["loss"].as_data_frame()) - shift, np.exp(predictions.as_data_frame()) - shift)
    if score < min_mae:
        min_mae = score
        
    print("MAE {}".format(score))
    return {'loss': score, 'status': STATUS_OK}


def optimize(trials):
    space = {
             'ntrees' : hp.choice('ntrees', [100, 250, 500, 750,1000,1250, 1500, 1750, 2000]),
             'learn_rate' : hp.quniform('learn_rate', 0.01, 5, 0.01),
             'max_depth' : hp.choice('max_depth', [100, 250, 500, 750,1000,1250, 1500, 1750, 2000,2250]),            
             'sample_rate' : hp.quniform('sample_rate', 0.01, 1, 0.01),
             'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
             'col_sample_rate' : hp.quniform('col_sample_rate', 0.01, 1, 0.01),
             }
    
    
    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=1000)
    print("Best params:")
    print(best)
    print("Min MAE={}".format(min_mae))


#Trials object where the history of search will be stored
trials = Trials()

optimize(trials)

Training with params : 
{'gamma': 0.65, 'sample_rate': 0.04, 'learn_rate': 4.62, 'max_depth': 100, 'ntrees': 750, 'col_sample_rate': 0.33}


H2OResponseError: Server error water.exceptions.H2OKeyNotFoundArgumentException:
  Error: Object 'None' not found in function: predict for argument: model
  Request: POST /4/Predictions/models/None/frames/Key_Frame__upload_8fe6e9bca7ea6ddf6fe559dff4ec4da1.hex
